In [1]:
import pandas as pd
import numpy as np
import os

from itertools import product

In [2]:
def trata_dados(df, ano, pop_name):
    df['ID_Municipio'] = df['Município'].str[:6] #coluna do codigo do municipio
    df['Município'] = df['Município'].str[7:] #coluna do nome do municipio
    df['ano'] = ano #coluna do ano dos dados

    df.columns = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total', 'ID_Municipio', 'ano']
        
    df['ID_Municipio'] = pd.to_numeric(df['ID_Municipio'])
        
    #nao sei explicar direito o que faz, mas deixa tudo no formato desejado
    df = df.melt(id_vars=['Município', 'ID_Municipio', 'ano'], var_name='faixa_etaria', value_name=pop_name)
    
    return df

In [3]:
def gerar_df(cols, csv_name, value_name):
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')

    padrao = df.iloc[[-1]]
    padrao.columns = cols

    padrao = padrao.melt(id_vars=['Município'], var_name='faixa_etaria', value_name=value_name)
    padrao.drop(labels=['Município'], axis=1, inplace=True)
    
    return df, padrao

In [4]:
#gera o df com a populacao total padrao de 2010

cols = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total']

#gera o df com a populacao total padrao de 2010
df, padrao_t = gerar_df(cols, "dados total/2010total", "pop_pad_total")

#gera o df com a populacao masculina padrao de 2010
df, padrao_h = gerar_df(cols, "dados homem/homem2010", "pop_pad_homem")

#gera o df com a populacao feminina padrao de 2010
df, padrao_m = gerar_df(cols, "dados mulher/mulher2010", "pop_pad_mulher")


In [5]:
padrao_t

,faixa_etaria,pop_pad_total
0,0a4,1338553
1,5a9,1497792
2,10a14,1711110
3,15a19,1742648
4,20a24,1757058
5,25a29,1727867
6,30a34,1617371
7,35a39,1435712
8,40a44,1392455
9,45a49,1312183


In [6]:
def gera_df_e_remove_linha(csv_name, title):
    #populacao total
    df = pd.read_csv(f"dados populacionais/{csv_name}.csv", encoding='latin1', sep=';')
    df.drop(df.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

    df = trata_dados(df, 2010, title)
    return df

#populacao total
df = gera_df_e_remove_linha("dados total/2010total", "pop_total")

#populacao masculina
df_h = gera_df_e_remove_linha("dados homem/homem2010", "pop_homem")

#populacao feminina
df_m = gera_df_e_remove_linha("dados mulher/mulher2010", "pop_mulher")


In [7]:
def pegar_arquivos(df, pasta, csv_name, title):
    directory = os.fsencode(f"./dados populacionais/{pasta}/") #populacao total

    anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

    i = 0

    #cria o dataframe com populacao total
    for file in os.listdir(directory):
        filename = os.fsdecode(file)

        if filename != f'{csv_name}.csv':
            aux = pd.read_csv(f"./dados populacionais/{pasta}/" + filename, encoding='latin1', sep=";")
            aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

            aux = trata_dados(aux, anos[i], title)

            df = df.append(aux)

            i = i + 1 #avanca o index de anos
            
    return df
            
df = pegar_arquivos(df, "dados total", "2010total", "pop_total")
df_h = pegar_arquivos(df_h, "dados homem", "homem2010", "pop_homem")
df_m = pegar_arquivos(df_m, "dados mulher", "mulher2010", "pop_mulher")

In [8]:
def unir_dfs(df1, df2, padrao2):
    df1 = df1.merge(df2, how="inner")
    df1 = df1.merge(padrao2, how="inner")
    return df1

df = df.merge(padrao_t, how="inner")

df = unir_dfs(df, df_h, padrao_h)
df = unir_dfs(df, df_m, padrao_m)

df

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher
0,Abadia dos Dourados,310010,2010,0a4,418,1338553,211,684629,207,653924
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924
2,Abre Campo,310030,2010,0a4,914,1338553,464,684629,450,653924
3,Acaiaca,310040,2010,0a4,279,1338553,135,684629,144,653924
4,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924
...,...,...,...,...,...,...,...,...,...,...
153535,Virginópolis,317180,2019,Total,10510,19957444,5180,9828254,5330,10129190
153536,Virgolândia,317190,2019,Total,5380,19957444,2622,9828254,2758,10129190
153537,Visconde do Rio Branco,317200,2019,Total,42564,19957444,21194,9828254,21370,10129190
153538,Volta Grande,317210,2019,Total,5252,19957444,2512,9828254,2740,10129190


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153540 entries, 0 to 153539
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Município       153540 non-null  object
 1   ID_Municipio    153540 non-null  int64 
 2   ano             153540 non-null  int64 
 3   faixa_etaria    153540 non-null  object
 4   pop_total       153540 non-null  int64 
 5   pop_pad_total   153540 non-null  int64 
 6   pop_homem       153540 non-null  int64 
 7   pop_pad_homem   153540 non-null  int64 
 8   pop_mulher      153540 non-null  int64 
 9   pop_pad_mulher  153540 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 12.9+ MB


In [10]:
#usado para obter os nomes de todos os agravos e seus subgrupos
agrav = pd.read_csv("Planilha para o R - Planilha.csv")
agrav = agrav.drop(labels='Unnamed: 0', axis=1)
agrav = agrav.fillna(0)
agrav.columns = ['cidsize', 'cid', 'agravo', 'subgrupo']

#listas com o nomes das doenças, anos, e meses
nomes_agravo = np.concatenate((agrav['agravo'].unique(), agrav['subgrupo'].unique()[1:], 'total', 'naturais', 'externas'), axis=None)
faixas = df['faixa_etaria'].unique()
faixas = faixas[:-1]
anos = [ano for ano in range(2010, 2020)]
meses = [mes for mes in range(1, 13)]

In [11]:
#grupos de cidades analisadas
cidades_selecionadas = ["Betim", 'Brumadinho','Curvelo','Esmeraldas','Florestal','Fortuna de Minas',
                        'Igarapé','Juatuba','Maravilhas','Mário Campos','Martinho Campos',
                        'Papagaios','Pará de Minas','Paraopeba','Pequi','Pompéu','São Joaquim de Bicas',
                        'São José da Varginha','Sarzedo']

cidades_limitrofes = ['Abaeté','Araçaí','Belo Horizonte','Belo Vale','Bom Despacho','Bonfim','Cachoeira da Prata',
                      'Caetanópolis','Capim Branco','Conceição do Pará','Contagem','Cordisburgo','Corinto',
                      'Dores do Indaiá','Felixlândia','Ibirité','Igaratinga','Inhaúma','Inimutaba','Itaúna',
                      'Itabirito','Itatiaiuçu','Leandro Ferreira','Mateus Leme','Matozinhos','Moeda',
                      'Morada Nova de Minas','Morro da Garça','Nova Lima','Onça de Pitangui','Paineiras',
                      'Pedro Leopoldo','Pitangui','Presidente Juscelino','Quartel Geral','Ribeirão das Neves',
                      'Rio Manso','Santana de Pirapama','Santo Hipólito','Sete Lagoas']


cidades_prox_limitrofes = ['Araújos','Augusto de Lima','Baldim','Biquinhas','Carmo do Cajuru','Cedro do Abaeté',
                           'Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Crucilândia',
                           'Estrela do Indaiá','Funilândia','Gouveia','Itaguara','Jaboticatubas','Jeceaba',
                           'Jequitibá','Lagoa Santa','Lassance','Luz','Moema','Monjolos','Nova Serrana',
                           'Ouro Preto','Piedade dos Gerais','Prudente de Morais',
                           'Raposos','Rio Acima','Sabará','São Gonçalo do Abaeté','São Gonçalo do Pará',
                           'São Gotardo','São José da Lapa','Santa Bárbara','Santa Luzia','Santana do Riacho',
                           'Santo Antônio do Monte','Serra da Saudade','Tiros','Vespasiano']


#gera uma lista com o ID de todas as cidades a serem avaliadas
todas = cidades_selecionadas + cidades_limitrofes + cidades_prox_limitrofes

cidades = df[df['Município'].isin(todas)]['ID_Municipio'].unique()

In [18]:
todas

['Abaeté',
 'Araçaí',
 'Araújos',
 'Augusto de Lima',
 'Baldim',
 'Belo Horizonte',
 'Belo Vale',
 'Betim',
 'Biquinhas',
 'Bom Despacho',
 'Bonfim',
 'Brumadinho',
 'Cachoeira da Prata',
 'Caetanópolis',
 'Capim Branco',
 'Carmo do Cajuru',
 'Cedro do Abaeté',
 'Conceição do Mato Dentro',
 'Conceição do Pará',
 'Confins',
 'Congonhas',
 'Congonhas do Norte',
 'Contagem',
 'Cordisburgo',
 'Corinto',
 'Crucilândia',
 'Curvelo',
 'Dores do Indaiá',
 'Esmeraldas',
 'Estrela do Indaiá',
 'Felixlândia',
 'Florestal',
 'Fortuna de Minas',
 'Funilândia',
 'Gouveia',
 'Ibirité',
 'Igarapé',
 'Igaratinga',
 'Inhaúma',
 'Inimutaba',
 'Itabirito',
 'Itaguara',
 'Itatiaiuçu',
 'Itaúna',
 'Jaboticatubas',
 'Jeceaba',
 'Jequitibá',
 'Juatuba',
 'Lagoa Santa',
 'Lassance',
 'Leandro Ferreira',
 'Luz',
 'Maravilhas',
 'Martinho Campos',
 'Mateus Leme',
 'Matozinhos',
 'Moeda',
 'Moema',
 'Monjolos',
 'Morada Nova de Minas',
 'Morro da Garça',
 'Mário Campos',
 'Nova Lima',
 'Nova Serrana',
 'Onça de P

In [12]:
#cria um dataframe vazio, que armazenará todas as permutações de municipio, ano, mês e agravo
df_size = len(list(product(cidades, faixas, anos, meses, nomes_agravo)))
df_index = [i for i in range(df_size)]

agregados = pd.DataFrame({'id_municipio': pd.Series([], dtype='float32'),
                            'faixa_etaria': pd.Series([], dtype='str'),
                            'ano': pd.Series([], dtype='float32'),
                            'mes': pd.Series([], dtype='float32'),
                            'agravo': pd.Series([], dtype='str')},
                            index = df_index)

agregados

,id_municipio,faixa_etaria,ano,mes,agravo
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
14687995,NaN,NaN,NaN,NaN,NaN
14687996,NaN,NaN,NaN,NaN,NaN
14687997,NaN,NaN,NaN,NaN,NaN
14687998,NaN,NaN,NaN,NaN,NaN


In [13]:
k = 0

data = agregados.values

for row in product(cidades, faixas, anos, meses, nomes_agravo):
    data[k] = row

    k = k + 1

In [14]:
np.savetxt("agregados.csv", data, fmt='%s', header="ID_Municipio faixa_etaria ano mes agravo", comments='')

In [15]:
agregados = pd.read_csv("agregados.csv", sep=" ")
agregados

,ID_Municipio,faixa_etaria,ano,mes,agravo
0,310020,0a4,2010,1,colera
1,310020,0a4,2010,1,diarreia
2,310020,0a4,2010,1,odip
3,310020,0a4,2010,1,tubresp
4,310020,0a4,2010,1,tbsistnerv
...,...,...,...,...,...
14687995,317120,>=80anos,2019,12,outraspele
14687996,317120,>=80anos,2019,12,veneno
14687997,317120,>=80anos,2019,12,total
14687998,317120,>=80anos,2019,12,naturais


In [16]:
sim = pd.read_csv("sim_agravos.csv", encoding="latin1")

letras_xx = ['V', 'W', 'X', 'Y']

In [17]:
#obitos de agravos por faixa etaria municipio ano e mes
aux = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_total']

#adiciona os agravos considerados como "subgrupos" na tabela
aux_sub = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_total']

aux = aux.append(aux_sub, ignore_index=True)

aux_total = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
aux_total.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'obitos_total']

aux_total['agravo'] = 'total'
                       
aux = aux.append(aux_total, ignore_index=True)

sim_nat = sim[~sim['causa3'].str[0].isin(letras_xx)]
sim_xx = sim[sim['causa3'].str[0].isin(letras_xx)]

aux_nat = sim_nat[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
aux_nat.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'obitos_total']
aux_nat['agravo'] = 'naturais'

aux = aux.append(aux_nat, ignore_index=True)

aux_xx = sim_xx[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
aux_xx.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'obitos_total']
aux_xx['agravo'] = 'externas'

aux = aux.append(aux_xx, ignore_index=True)

aux

,ID_Municipio,faixa_etaria,ano,mes,agravo,obitos_total
0,310000,0a4,2010,3,homicidio,1
1,310000,0a4,2010,11,infecespperi,1
2,310000,0a4,2011,1,homicidio,1
3,310000,0a4,2011,6,exttrauma,1
4,310000,0a4,2016,4,extintind,1
...,...,...,...,...,...,...
1514371,317220,65a69,2012,1,externas,1
1514372,317220,65a69,2013,6,externas,1
1514373,317220,70a74,2013,1,externas,1
1514374,317220,75a79,2018,5,externas,1


In [18]:
def gerar_obitos(sim, sexo2, nome_obito):
    sim_aux = sim[sim['sexo2'] == sexo2]
    aux = sim_aux[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
    aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'obitos_{nome_obito}']

    aux_sub = sim_aux[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
    aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', f'obitos_{nome_obito}']

    aux = aux.append(aux_sub, ignore_index=True)
    
    aux_total = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
    aux_total.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', f'obitos_{nome_obito}']

    aux_total['agravo'] = 'total'

    aux = aux.append(aux_total, ignore_index=True)

    sim_nat = sim[~sim['causa3'].str[0].isin(letras_xx)]
    sim_xx = sim[sim['causa3'].str[0].isin(letras_xx)]

    aux_nat = sim_nat[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
    aux_nat.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', f'obitos_{nome_obito}']
    aux_nat['agravo'] = 'naturais'

    aux = aux.append(aux_nat, ignore_index=True)

    aux_xx = sim_xx[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes'], as_index=False).count()
    aux_xx.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', f'obitos_{nome_obito}']
    aux_xx['agravo'] = 'externas'

    aux = aux.append(aux_xx, ignore_index=True)
    
    return aux

In [19]:
aux_h = gerar_obitos(sim, 'M', "homem")
aux_m = gerar_obitos(sim, 'F', "mulher")

In [20]:
aux = aux.merge(aux_h, how='left')
aux = aux.merge(aux_m, how='left')

In [21]:
taxas = agregados.merge(aux, how='left')

taxas = taxas.fillna(0)
taxas = taxas.astype({'obitos_total': 'int32',
                        'obitos_homem': 'int32',
                        'obitos_mulher': 'int32'})
taxas

,ID_Municipio,faixa_etaria,ano,mes,agravo,obitos_total,obitos_homem,obitos_mulher
0,310020,0a4,2010,1,colera,0,0,0
1,310020,0a4,2010,1,diarreia,0,0,0
2,310020,0a4,2010,1,odip,0,0,0
3,310020,0a4,2010,1,tubresp,0,0,0
4,310020,0a4,2010,1,tbsistnerv,0,0,0
...,...,...,...,...,...,...,...,...
14687995,317120,>=80anos,2019,12,outraspele,0,0,0
14687996,317120,>=80anos,2019,12,veneno,0,0,0
14687997,317120,>=80anos,2019,12,total,13,13,13
14687998,317120,>=80anos,2019,12,naturais,13,13,13


In [22]:
def calcular_taxa_obitos(obitos, taxa_pop):
    return (obitos / taxa_pop) * 100000

def calcular_obitos_esp(pop_pad, taxa_obitos):
    return pop_pad * (taxa_obitos / 100000)

def calcular_taxa_obitos_esp(obitos_esp, taxa_pop_pad):
    return (obitos_esp / taxa_pop_pad) * 100000

def calcular_taxas(taxas):
    labels = ['total', 'homem', 'mulher']
    for t in labels:
        taxas[f'taxa_obitos_{t}']  = calcular_taxa_obitos(taxas[f'obitos_{t}'], taxas[f'pop_{t}'])
    
    for t in labels:
        taxas[f'obitos_esp_{t}']  = calcular_obitos_esp(taxas[f'pop_pad_{t}'], taxas[f'taxa_obitos_{t}'])
    
    for t in labels:
        taxas[f'taxa_obitos_esp_{t}']  = calcular_taxa_obitos_esp(taxas[f'obitos_esp_{t}'], taxas[f'pop_pad_{t}'])
    
    return taxas

In [23]:
taxas = df.merge(taxas, how='inner')

taxas = calcular_taxas(taxas)

In [24]:
taxas['grupo'] = np.nan

taxas.loc[(taxas['Município'].isin(cidades_selecionadas)), 'grupo'] = 1
taxas.loc[(taxas['Município'].isin(cidades_limitrofes)), 'grupo'] = 2
taxas.loc[(taxas['Município'].isin(cidades_prox_limitrofes)), 'grupo'] = 3

taxas = taxas[~taxas['grupo'].isnull()]

In [25]:
taxas

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_obitos_total,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher,grupo
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14687995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
14687996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
14687997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,862.641009,2462.121212,1327.885598,2885.810219,3186.748106,2723.506639,862.641009,2462.121212,1327.885598,3.0
14687998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,862.641009,2462.121212,1327.885598,2885.810219,3186.748106,2723.506639,862.641009,2462.121212,1327.885598,3.0


In [26]:
taxas.columns

Index(['Município', 'ID_Municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'obitos_total', 'obitos_homem',
       'obitos_mulher', 'taxa_obitos_total', 'taxa_obitos_homem',
       'taxa_obitos_mulher', 'obitos_esp_total', 'obitos_esp_homem',
       'obitos_esp_mulher', 'taxa_obitos_esp_total', 'taxa_obitos_esp_homem',
       'taxa_obitos_esp_mulher', 'grupo'],
      dtype='object')

In [27]:
taxas.columns = ['municipio', 'id_municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'obitos_total',
       'obitos_homem', 'obitos_mulher', 'taxa_obitos_total',
       'taxa_obitos_homem', 'taxa_obitos_mulher', 'obitos_esp_total',
       'obitos_esp_homem', 'obitos_esp_mulher', 'taxa_obitos_esp_total',
       'taxa_obitos_esp_homem', 'taxa_obitos_esp_mulher', 'grupo']

taxas

,municipio,id_municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_obitos_total,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher,grupo
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14687995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
14687996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
14687997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,862.641009,2462.121212,1327.885598,2885.810219,3186.748106,2723.506639,862.641009,2462.121212,1327.885598,3.0
14687998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,862.641009,2462.121212,1327.885598,2885.810219,3186.748106,2723.506639,862.641009,2462.121212,1327.885598,3.0


In [28]:
taxas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14688000 entries, 0 to 14687999
Data columns (total 25 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   municipio               object 
 1   id_municipio            int64  
 2   ano                     int64  
 3   faixa_etaria            object 
 4   pop_total               int64  
 5   pop_pad_total           int64  
 6   pop_homem               int64  
 7   pop_pad_homem           int64  
 8   pop_mulher              int64  
 9   pop_pad_mulher          int64  
 10  mes                     int64  
 11  agravo                  object 
 12  obitos_total            int32  
 13  obitos_homem            int32  
 14  obitos_mulher           int32  
 15  taxa_obitos_total       float64
 16  taxa_obitos_homem       float64
 17  taxa_obitos_mulher      float64
 18  obitos_esp_total        float64
 19  obitos_esp_homem        float64
 20  obitos_esp_mulher       float64
 21  taxa_obitos_esp_total   float

In [29]:
taxas.to_csv('pre_taxa_sim.csv', index=False)